In [2]:
import numpy as np
import pandas as pd
import xarray as xr
import glob

In [8]:
filedir_path = "../../data/raw/btsdelay/"
files = glob.glob(f"{filedir_path}*.parquet")

# df = pd.concat([pd.read_parquet(i) for i in files]).sample(1_000_000)
raw = pd.read_parquet(files[0]).sample(100_000)


In [9]:
raw.describe()

,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,ArrTime,ArrDelayMinutes,AirTime,CRSElapsedTime,ActualElapsedTime,Distance,...,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,DistanceGroup,DivAirportLandings
count,100000.000000,98469.000000,98448.000000,98448.000000,98388.000000,98154.000000,98044.000000,100000.000000,98163.000000,100000.000000,...,98304.000000,98304.000000,98268.000000,98268.000000,100000.000000,98154.000000,98154.000000,98154.000000,100000.000000,100000.000000
mean,1330.714500,1334.252171,13.554709,10.239477,1463.544009,13.759938,109.855595,138.605070,134.133105,785.355330,...,17.056396,1358.013173,1459.451978,7.353513,1484.457640,5.504126,0.194225,-0.155979,3.616550,0.003300
std,490.451201,504.479397,44.766723,45.925607,540.128040,44.625055,70.354234,72.543215,72.326416,592.339033,...,9.843895,505.874544,535.984260,5.766518,519.708214,48.026855,0.395605,2.365358,2.339105,0.095442
min,1.000000,1.000000,0.000000,-199.000000,1.000000,0.000000,9.000000,-36.000000,12.000000,16.000000,...,1.000000,1.000000,1.000000,0.000000,1.000000,-194.000000,0.000000,-2.000000,1.000000,0.000000
25%,915.000000,916.000000,0.000000,-6.000000,1047.000000,0.000000,59.000000,85.000000,81.000000,349.000000,...,11.000000,931.000000,1043.000000,4.000000,1100.000000,-14.000000,0.000000,-1.000000,2.000000,0.000000
50%,1325.000000,1328.000000,0.000000,-2.000000,1506.000000,0.000000,91.000000,120.000000,116.000000,626.000000,...,15.000000,1342.000000,1502.000000,6.000000,1515.000000,-6.000000,0.000000,-1.000000,3.000000,0.000000
75%,1735.000000,1745.000000,7.000000,7.000000,1915.000000,8.000000,139.000000,169.000000,164.000000,1023.000000,...,20.000000,1800.000000,1911.000000,9.000000,1918.000000,8.000000,0.000000,0.000000,5.000000,0.000000
max,2359.000000,2400.000000,2098.000000,2098.000000,2400.000000,2108.000000,635.000000,690.000000,680.000000,4983.000000,...,196.000000,2400.000000,2400.000000,129.000000,2400.000000,2108.000000,1.000000,12.000000,11.000000,9.000000


In [13]:
raw.columns.tolist()[:20]

['FlightDate',
 'Airline',
 'Origin',
 'Dest',
 'Cancelled',
 'Diverted',
 'CRSDepTime',
 'DepTime',
 'DepDelayMinutes',
 'DepDelay',
 'ArrTime',
 'ArrDelayMinutes',
 'AirTime',
 'CRSElapsedTime',
 'ActualElapsedTime',
 'Distance',
 'Year',
 'Quarter',
 'Month',
 'DayofMonth']

In [40]:
squared_delays = (df
                  .assign(sqrd_delay = lambda df_: df_['ArrDelay']**2)
                  [['Origin', 'Dest', 'sqrd_delay']]
                  .groupby(['Origin', 'Dest'])
                  .agg(mean_sqr_delay = pd.NamedAgg('sqrd_delay', np.sum),)
                  .reset_index()
                  )
squared_delays.head().values.tolist()

[['ABE', 'ATL', 390552.0],
 ['ABE', 'CLT', 140668.0],
 ['ABE', 'DTW', 329615.0],
 ['ABE', 'FLL', 72626.0],
 ['ABE', 'MYR', 448168.0]]

In [74]:
def construct_adj_mat(flights: pd.DataFrame) -> pd.DataFrame:
    """ajd matrix of form Aij => flight from i to j, id est rows are origin, cols are dest """

    squared_delays = (flights
                      .assign(sqrd_delay = lambda df_: df_['ArrDelay']**2,
                              dtime=lambda df_: pd.to_datetime((df_['FlightDate'].astype(str) + " " +
                                                                df_['CRSDepTime'].astype(int).astype(str)),
                                    format="%Y-%m-%d %H%M%S", errors='coerce'))
                      [['Origin', 'Dest', 'sqrd_delay', 'dtime']]
                      .groupby(['Origin', 'Dest', pd.Grouper(key='dtime', freq="H")])
                      .agg(mean_sqr_delay = pd.NamedAgg('sqrd_delay', np.sum),)
                      .reset_index()
                      )

    locs = np.unique(np.concatenate((flights['Origin'].unique(), flights['Dest'].unique())))
    
    time_grouped = (squared_delays
                    .groupby(pd.Grouper(key="dtime", freq="H")))

    print(time_grouped.head())
    adj_mat = xr.DataArray(np.zeros((len(locs), len(locs), len(time_grouped)), dtype=np.single),
                           dims=("Origin", "Dest", "timestamp"),
                           coords={"Origin": locs, 'Dest': locs, 'timestamp': [t for t, _ in time_grouped]})
    
    for _, group in time_grouped:
        for idx, origin, dest, dtime, mean_sqr_delay in group.itertuples():
            print(origin, dest, dtime)
            adj_mat.loc[{'Origin': origin, 'Dest': dest, 'timestamp': dtime}] = mean_sqr_delay

    return adj_mat


adj_mat = construct_adj_mat(raw)
adj_mat

      Origin Dest               dtime  mean_sqr_delay
5777     AUS  LAS 2018-01-01 05:00:00             4.0
34890    FLL  BOS 2018-01-01 05:00:00            36.0
49732    LAS  SAN 2018-01-01 05:00:00             0.0
59993    MDW  BWI 2018-01-01 05:00:00           121.0
81182    RDM  SFO 2018-01-01 05:00:00           324.0
...      ...  ...                 ...             ...
35447    FLL  IAG 2018-12-31 22:00:00           400.0
41394    IAD  TPA 2018-12-31 22:00:00           625.0
88582    SEA  TPA 2018-12-31 22:00:00           256.0
48335    LAS  CMH 2018-12-31 23:00:00           400.0
67503    OGG  HNL 2018-12-31 23:00:00           169.0

[33145 rows x 4 columns]
AUS LAS 2018-01-01 05:00:00
FLL BOS 2018-01-01 05:00:00
LAS SAN 2018-01-01 05:00:00
MDW BWI 2018-01-01 05:00:00
RDM SFO 2018-01-01 05:00:00
ABQ LAX 2018-01-01 06:00:00
ALB MSP 2018-01-01 06:00:00
BOS MCO 2018-01-01 06:00:00
CLT DTW 2018-01-01 06:00:00
DAL DEN 2018-01-01 06:00:00
DEN ATL 2018-01-01 06:00:00
DHN ATL 2018-01-01

In [67]:
(raw
                      .assign(sqrd_delay = lambda df_: df_['ArrDelay']**2,
                              dtime=lambda df_: pd.to_datetime((df_['FlightDate'].astype(str) + 
                                                                df_['CRSDepTime'].astype(int).astype(str)),
                                    format="%Y-%m-%d %H%M%S", errors='coerce'))
                      [['Origin', 'Dest', 'sqrd_delay', 'dtime']]
                      .dtypes
                      #.groupby(['Origin', 'Dest', 'dtime'])
                      #.agg(mean_sqr_delay = pd.NamedAgg('sqrd_delay', np.mean),)
                      #.reset_index()
                      )

Origin                object
Dest                  object
sqrd_delay           float64
dtime         datetime64[ns]
dtype: object

In [69]:
adj_mat.to_dataset(name='name')

<xarray.Dataset>
Dimensions:  (Origin: 370, Dest: 370)
Coordinates:
  * Origin   (Origin) object 'ABE' 'ABI' 'ABQ' 'ABR' ... 'YAK' 'YKM' 'YNG' 'YUM'
  * Dest     (Dest) object 'ABE' 'ABI' 'ABQ' 'ABR' ... 'YAK' 'YKM' 'YNG' 'YUM'
Data variables:
    name     (Origin, Dest) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0